# EVM Symbolic Execution

Our goal is to add symbolic execution to a rust based implementation of the EVM.

## Symbolic execution in the large
Symbolic execution is a general (not EVM specific) program analysis technique.
- [Symbolic Execution Introduction](https://en.wikipedia.org/wiki/Symbolic_execution) read the intro paragraph and the example.
- [Symbolic Execution Lecture](https://www.youtube.com/watch?v=yRVZPvHYHzw) watch the whole thing. You should follow most all of it before you continue.

Symbolic execution also has applications to the EVM. It can be used to find conditions that lead to assertion violations as well as other use cases. 
- [HEVM Symbolic Execution](https://fv.ethereum.org/2020/07/28/symbolic-hevm-release/) read the full post but do not attempt to understand everything.
- [Dapptools testing integration](https://youtu.be/N9pJ9JieX10?t=990) Watch the linked timestamp to see how dapptools uses hevm to symbolically search for assertion failures

All symbolic execution implementations rely on an SMT Solver. You should have a basic understanding of the SMT Solver's role from the Symbolic Execution Lecture. We will be using Z3 for our examples because it has a nice python library. For our actual implementation we will be as solver agnostic as possible.
- [Z3 tutorial](https://theory.stanford.edu/~nikolaj/programmingz3.html) Read all of section 2 and sections 3.1-3.4. We will not be too deep in the weeds of the solver so focus on intuition instead of formal understanding. Feel free to play with some of the code samples.

## Vocabulary
- Concrete value - A value that is known at runtime. When concrete values are combined with other concrete values, the result is also concrete.
- Symbolic value - A value that is not known at runtime. Symbolic values still have known and fixed types. When symbolic values are combined with both concrete and symbolic values, the result is also symbolic.
- Memory model - How memory is represented so that it can hold symbolic values. "Memory" in the general sense of all readable and writable storage locations. I.e. for the EVM, the memory model refers to memory, storage, calldata, and code.
- Concrete instruction - An instruction that only operates on concrete values.
- Symbolic Instruction - An instruction that can operates on symbolic values, concrete values, and combinations of the two.
- (Path) Constraint set - The set of constraints collected while executing the program. They constrain the set of possible concrete values that the symbolic values can represent.
- Machine state - Abstract term referring to the current state of (a single) execution of the program. This includes the state of the memory mode, the constraint set, program counter, etc...
- Interpreter - The function that takes in one machine state, executes an instruction, and returns multiple machine states. If the returning multiple machine states part confuses you, that's ok, assume it returns a single updated machine state for now.
- Runtime - Abstract term referring to the code orchestrating the memory model, interpreter, and machine state to compute.
- Solver - The SMT solver that is used to check if constraints can be satisfied.
- Satisfied - There exists a possible solution to the set of constraints given. A solution can be used to replace symbolic values with concrete values that would lead to the current machine state.
- Unsatisfied - The solver could not find a solution to the set of constraints given. An unsatisfied set of constraints is used to assume that the current machine state cannot exist and it should not be explored further. Note that this doesn't mean that one doesn't exist and can be a false negative.


## Symbolic execution in the small
We are going to walk through some examples of implementing symbolic execution against an EVM-like assembly language. 

All symbolic execution implementations follow a common pattern of taking an existing program runtime that executes over concrete values and modifying it such that...
- All concrete readable and writable values are replaced with a memory model that handles symbolic values.
- The interpreter executes symbolic instructions
- The machine state tracks path constraints
- The runtime can pass a set of path constraints to a solver to check for satisfiability
- The Runtime can handle multiple concurrent machine states (ok if you don't understand why right now)

We are going to work our way up from simpler machines to more complex machines adding incremental symbolic execution functionality as needed.

One important thing to note is that there is not one way to implement symbolic execution and symbolic execution is not all or nothing. You can force parts of the runtime to only execute over concrete values and in fact frequently we do for performance reasons.

Note that all python code runs on Python 3.10 and we use the library [z3-solver](https://pypi.org/project/z3-solver/) to interact with Z3.

### Arithmetic operations on stack values
Our first example will be assuming that our language has ADD and SUB instructions that operate on the stack. For simplicity sake, we will assume that we can store actual integers on the stack and not fixed width bit representations of integers. We consider the "result" of the program to be the item on the top of the stack when the program ends. Take this simple implementation.

In [27]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int

class StackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

We can compute things!

In [24]:
# (3 + 2) - 1 -> 4
m = StackArith([PUSH(1), PUSH(2), PUSH(3), ADD(), SUB()])
m.run()

4

Now what happens when we put symbolic values on the stack. Note that we don't have to change the implementation. This is because the z3 objects implement their own `+` method.

In [29]:
from z3 import *
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
m.run()

z + y - x

The result of the computation is also symbolic. Note that the z3 library prints a nice string representation of the compound symbolic value. The actual type of the python value is from the Z3 package and the sort is still int. "Sorts"  are the Z3 equivalent of type (is this actually true?). Really "sort" comes from smtlib and not Z3 but that's getting ahead of ourselves.

In [32]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB()])
res = m.run()
print(res.__class__)
print(res.sort())

<class 'z3.z3.ArithRef'>
Int


We can also include concrete values on the stack, so the returned value contains both symbolic and concrete values.

In [33]:
m = StackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(3), ADD(), SUB()])
m.run()

3 + y - x

Just computing on symbolic values can be helpful in certain circumstances, but we want to be able to ask the question "what inputs do I have to pass this program for it to give this output". Let's add an additional instruction `ASSERT(n)` that adds the constraint that the top item on the stack is equal to its argument.

In [64]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class ASSERT:
    n: int

class SymStackArith:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX We add the set of constraints to the machine state
        # Solver is the object from the z3 package that can be used
        # to hold constraints
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case ASSERT(n):
                # XXX We add the constraint to the machine state.
                self.constraints.add(self.stack[-1] == n)
            case _:
                assert False
            
    # Returns: (The symbolic value, the concrete value (if satisfiable), the model (if satisfiable))
    def run(self):
        while self.pc < len(self.code):
            self.step()
            
        # XXX We check if the set of constraints is satisfiable.
        rv = self.stack[-1]
        ans = Int('ANS')
        self.constraints.add(ans == rv)
        if self.constraints.check() == sat:
            model = self.constraints.model()
            return rv, model[ans], model
        else:
            # The set of constraints is not satisfiable. There is no
            # model or concrete value to return
            return rv, None, None

In [65]:
m = SymStackArith([PUSH(Int('x')), PUSH(Int('y')), PUSH(Int('z')), ADD(), SUB(), ASSERT(5)])
m.run()

(z + y - x, 5, [ANS = 5, y = 0, x = 0, z = 5])

The answer here is not terribly interesting, but it tells us that replacing x, y, and z with 0, 0, and 5 respectively will result in the final value on the stack being equal to 5. We can confirm this by re-running the same machine with the concrete values.

In [66]:
m = SymStackArith([PUSH(0), PUSH(0), PUSH(5), ADD(), SUB()])
m.run()

(5, 5, [ANS = 5])

What happens when the solution is not satisfiable? Let's try a basic example where we assert one symbolic value to be equal to two different concrete values

In [67]:
m = SymStackArith([PUSH(Int('x')), ASSERT(1), ASSERT(2)])
m.run()

(x, None, None)

The constraint solver tells us that no possible concret value of `x` can satisfy both constraints.

### Read only memory


In [6]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

class ReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

In [7]:
mem = [0 for x in range(5)]
mem[4] = 3
# (mem[4] + 2) - 1
# -> (3 + 2) - 1 
# -> 4
m = ReadonlyMem([PUSH(1), PUSH(2), PUSH(4), MLOAD(), ADD(), SUB()], mem)
m.run()

4

In [11]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        # XXX
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                idx = self.stack.pop()
                assert isinstance(idx, int), 'MLOAD: requires concrete index'
                self.stack.append(self.memory[idx])
            case ASSERT(n):
                # XXX
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        # XXX
        rv = self.stack[-1]
        ans = Int('ANS')
        self.constraints.add(ans == rv)
        if self.constraints.check() == sat:
            model = self.constraints.model()
            return rv, model[ans], model
        else:
            return rv, None, None

In [13]:
mem = [0 for x in range(5)]
mem[4] = Int('z')
m = SymReadonlyMem([PUSH(Int('x')), PUSH(Int('y')), PUSH(4), MLOAD(), ADD(), SUB(), ASSERT(5)], mem)
m.run()

(z + y - x, 5, [ANS = 5, y = 0, x = 0, z = 5])

In [22]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class ASSERT:
    n: int

class SymReadonlyMemSymIndex:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        # XXX
        self.memory = Function('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                # XXX
                self.stack.append(self.memory(self.stack.pop()))
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        ans = Int('ANS')
        self.constraints.add(ans == rv)
        if self.constraints.check() == sat:
            model = self.constraints.model()
            return rv, model[ans], model
        else:
            return rv, None, None

In [23]:
m = SymReadonlyMemSymIndex([PUSH(Int('x')), PUSH(Int('y')), PUSH(4), MLOAD(), ADD(), SUB(), ASSERT(5)])
m.run()

(memory(4) + y - x, 5, [ANS = 5, x = 0, y = 5, memory = [else -> 0]])

In [33]:
m = SymReadonlyMemSymIndex([
    # x
    PUSH(0), MLOAD(),
    # y
    PUSH(1), MLOAD(),
    # z
    PUSH(2), MLOAD(),
    ADD(),
    # (y + z) == 5
    ASSERT(5),
    SUB(),
    # (y + z) - x == 4
    ASSERT(4)
])
m.run()

(memory(2) + memory(1) - memory(0),
 4,
 [ANS = 4, memory = [1 -> 0, 0 -> 1, else -> 5]])

### Read/Write memory

In [70]:
from dataclasses import dataclass

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

class RWMem:
    def __init__(self, code, memory):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = memory
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory[self.stack.pop()])
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()
                self.memory[idx] = val
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()
        return self.stack[-1]

In [71]:
mem = [0 for x in range(5)]

m = RWMem([
    PUSH(1),
    PUSH(2),
    PUSH(3), PUSH(0), MSTORE(),
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
], mem)

m.run()

4

In [72]:
from dataclasses import dataclass
from z3 import *

@dataclass
class ADD:
    pass

@dataclass
class SUB:
    pass

@dataclass
class PUSH:
    n: int
        
@dataclass
class MLOAD:
    pass

@dataclass
class MSTORE:
    pass

@dataclass
class ASSERT:
    n: int

class SymRWMem:
    def __init__(self, code):
        self.pc = 0
        self.code = code
        self.stack = []
        self.memory = Function('memory', IntSort(), IntSort())
        self.constraints = Solver()
        
    def step(self):
        instruction = self.code[self.pc]
        self.pc += 1
        match instruction:
            case ADD():
                self.stack.append(self.stack.pop() + self.stack.pop())
            case SUB():
                self.stack.append(self.stack.pop() - self.stack.pop())
            case PUSH(n):
                self.stack.append(n)
            case MLOAD():
                self.stack.append(self.memory(self.stack.pop()))
            case MSTORE():
                # XXX
                idx = self.stack.pop()
                val = self.stack.pop()                
                self.constraints.add(self.memory(idx) == val)
            case ASSERT(n):
                self.constraints.add(self.stack[-1] == n)                
            case _:
                assert False
                
    def run(self):
        while self.pc < len(self.code):
            self.step()

        rv = self.stack[-1]
        ans = Int('ANS')
        self.constraints.add(ans == rv)
        if self.constraints.check() == sat:
            model = self.constraints.model()
            return rv, model[ans], model
        else:
            return rv, None, None

In [74]:
m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('x')), PUSH(0), MSTORE(),
    PUSH(0), MLOAD(),
    ADD(), 
    SUB(), 
    ASSERT(4)
])
m.run()

(memory(0) + 2 - 1, 4, [ANS = 4, x = 3, memory = [else -> 3]])

In [75]:
idx = Int('idx')

m = SymRWMem([
    PUSH(1),
    PUSH(2),
    PUSH(Int('val')), PUSH(idx), MSTORE(),
    PUSH(idx), MLOAD(),
    ADD(), 
    SUB(),
    ASSERT(4)
])

m.constraints
m.run()

(memory(idx) + 2 - 1, 4, [idx = 0, ANS = 4, val = 3, memory = [else -> 3]])

### Conditionals
### Conditional Jumps
### A more realistic model of memory using Bit Vectors